<a href="https://colab.research.google.com/github/raulbenitez/DAPR/blob/master/Titanic_imputation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load titanic dataset from seaborn


In [1]:
import seaborn as sns
dtitanic = sns.load_dataset('titanic')
dtitanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


Check summary stats

In [2]:
dtitanic.describe()

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Find how many passangers have a missing age

In [3]:
dtitanic.loc[dtitanic['age'].isna()==True]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
17,1,2,male,NaN,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True
19,1,3,female,NaN,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True
26,0,3,male,NaN,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True
28,1,3,female,NaN,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,NaN,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,no,True
863,0,3,female,NaN,8,2,69.5500,S,Third,woman,False,NaN,Southampton,no,False
868,0,3,male,NaN,0,0,9.5000,S,Third,man,True,NaN,Southampton,no,True
878,0,3,male,NaN,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True


In [5]:
N_noage = dtitanic.loc[dtitanic['age'].isna()==True].shape[0]
print('There are {} passangers with unknown age'.format(N_noage))

There are 177 passangers with unknown age


We first keep the numerical variables only, removing all the others

In [6]:
dtitanic_num = dtitanic.select_dtypes(include='number')
dtitanic_num.head()

,survived,pclass,age,sibsp,parch,fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


Apply the KNN imputer from sklearn

In [9]:
import pandas as pd

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2, weights="uniform")
imputed_data = imputer.fit_transform(dtitanic_num)

df1_imputed = pd.DataFrame(imputed_data,columns=dtitanic_num.columns)
df1_imputed.loc[dtitanic_num['age'].isna()==True]

,survived,pclass,age,sibsp,parch,fare
5,0.0,3.0,26.50,0.0,0.0,8.4583
17,1.0,2.0,33.25,0.0,0.0,13.0000
19,1.0,3.0,18.50,0.0,0.0,7.2250
26,0.0,3.0,37.75,0.0,0.0,7.2250
28,1.0,3.0,24.00,0.0,0.0,7.8792
...,...,...,...,...,...,...
859,0.0,3.0,25.25,0.0,0.0,7.2292
863,0.0,3.0,45.50,8.0,2.0,69.5500
868,0.0,3.0,22.00,0.0,0.0,9.5000
878,0.0,3.0,28.00,0.0,0.0,7.8958


Now compare with the Iterative Imputer (sklearn)

In [10]:
import pandas as pd

# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer

imputer2 = IterativeImputer(random_state=0)
imputed_data = imputer2.fit_transform(dtitanic_num)

df1_imputed = pd.DataFrame(imputed_data,columns=dtitanic_num.columns)
df1_imputed.loc[dtitanic_num['age'].isna()==True]

,survived,pclass,age,sibsp,parch,fare
5,0.0,3.0,29.092853,0.0,0.0,8.4583
17,1.0,2.0,30.418546,0.0,0.0,13.0000
19,1.0,3.0,22.672326,0.0,0.0,7.2250
26,0.0,3.0,29.110029,0.0,0.0,7.2250
28,1.0,3.0,22.663215,0.0,0.0,7.8792
...,...,...,...,...,...,...
859,0.0,3.0,29.109970,0.0,0.0,7.2292
863,0.0,3.0,-5.074562,8.0,2.0,69.5500
868,0.0,3.0,29.078346,0.0,0.0,9.5000
878,0.0,3.0,29.100687,0.0,0.0,7.8958
